In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [80]:
df = pd.read_pickle(r'modeling-df.pickle')

In [81]:
df.head()

,perobs,perstop,explnstp,othpers,offunif,frisked,searched,contrabn,pistol,riflshot,asltweap,knifcuti,machgun,othrweap,pf_hands,pf_wall,pf_grnd,pf_drwep,pf_ptwep,pf_baton,pf_hcuff,pf_pepsp,pf_other,ac_rept,ac_inves,rf_vcrim,rf_othsw,ac_proxm,rf_attir,cs_objcs,cs_descr,cs_casng,cs_lkout,rf_vcact,cs_cloth,cs_drgtr,ac_evasv,ac_assoc,cs_furtv,rf_rfcmp,ac_cgdir,rf_verbl,cs_vcrim,cs_bulge,cs_other,ac_incid,ac_time,rf_knowl,ac_stsnd,ac_other,sb_hdobj,sb_outln,sb_admis,sb_other,rf_furt,rf_bulg,age,weight,ht_inches,target,pct_5,pct_6,pct_7,pct_9,pct_10,pct_13,pct_14,pct_17,pct_18,pct_19,pct_20,pct_22,pct_23,pct_24,pct_25,pct_26,pct_28,pct_30,pct_32,pct_33,pct_34,pct_40,pct_41,pct_42,pct_43,pct_44,pct_45,pct_46,pct_47,pct_48,pct_49,pct_50,pct_52,pct_60,pct_61,pct_62,pct_63,pct_66,pct_67,pct_68,...,pct_120,pct_121,pct_122,pct_123,inout_Outside,trhsloc_Neither,trhsloc_Transit,typeofid_P,typeofid_R,typeofid_V,sex_Male,sex_Other,race_ASIAN/PACIFIC ISLANDER,race_BLACK,race_BLACK-HISPANIC,race_OTHER,race_UNKNOWN,race_WHITE,race_WHITE-HISPANIC,build_M,build_T,build_U,build_Z,city_BROOKLYN,city_MANHATTAN,city_QUEENS,city_STATEN IS,month_August,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September,descr_AGGRAVATED HARASSMENT,descr_ARSON,descr_ASSAULT,descr_AUTO STRIPPING,descr_BIGAMY,descr_BRIBE RECEIVING,descr_BURGLARY,descr_COERCION,descr_COMPUTER TRESPASS,descr_COURSE OF SEXUAL CONDUCT,descr_CPSP,descr_CPW,descr_CREATING A HAZARD,descr_CRIMINAL CONTEMPT,descr_CRIMINAL MISCHIEF,descr_CRIMINAL POSSESION OF CONTROLLED SUBSTANCE,descr_CRIMINAL POSSESSION OF COMPUTER MATERIAL,descr_CRIMINAL POSSESSION OF FORGED INSTRUMENT,descr_CRIMINAL POSSESSION OF MARIHUANA,descr_CRIMINAL SALE OF CONTROLLED SUBSTANCE,descr_CRIMINAL SALE OF MARIHUANA,descr_CRIMINAL TAMPERING,descr_CRIMINAL TRESPASS,descr_ENDANGER THE WELFARE OF A CHILD,descr_ESCAPE,descr_FORCIBLE TOUCHING,descr_FORGERY,descr_FRAUD,descr_FRAUDULENT ACCOSTING,descr_GAMBLING,descr_GRAND LARCENY,descr_GRAND LARCENY AUTO,descr_HARASSMENT,descr_JOSTLING,descr_KIDNAPPING,descr_LOITERING,descr_MAKING GRAFFITI,descr_MENACING,descr_MISAPPLICATION OF PROPERTY,descr_MURDER,descr_OBSTRUCTING GOVERNMENTAL ADMINISTRATION,descr_OTHER,descr_PETIT LARCENY,descr_POSSESSION OF BURGLAR TOOLS,descr_POSSESSION OF GRAFFITI INSTRUMENTS,descr_PROHIBITED USE OF WEAPON,descr_PROSTITUTION,descr_PUBLIC LEWDNESS,descr_RAPE,descr_RECKLESS ENDANGERMENT,descr_REFUSING TO AID A PEACE OR POLICE OFFICER,descr_REWARD OFFICIAL MISCONDUCT,descr_ROBBERY,descr_SEXUAL ABUSE,descr_SEXUAL MISCONDUCT,descr_SEXUAL PERFORMANCE BY A CHILD,descr_TERRORISM,descr_THEFT OF SERVICES,descr_UNAUTHORIZED RECORDING,descr_UNAUTHORIZED USE OF A VEHICLE,descr_UNLAWFUL ASSEMBLY,descr_UNLAWFULL IMPRISONMENT
0,1,8,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,48,180,74,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8,10,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,20,150,68,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,10,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,34,140,66,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Preparing the dataset for modeling

In [166]:
# Split the df into target df and feature df
df_target = df.target
df_features = df.drop(['target'], axis=1)

In [167]:
df_features = df_features.astype(int)

## Logistic Regression Models

In [168]:
# Creating our train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_features, df_target, random_state=1, test_size=0.2)

In [169]:
# Normalize our data using min-max scaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Instatiate the scaler
scaler = MinMaxScaler()  

# Fit the scaler to our training feature set
scaler.fit(X_train)

# Using the fitted scaler, transform the training feature set to values between 0 and 1
X_train = scaler.transform(X_train)  

# Transform the testing set as well to ensure consistency in predictions across feature sets
X_test = scaler.transform(X_test) 

In [271]:
# Create list to house performance metrics
performance = []

In [272]:
# Fitting our basic logistic regression model.
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

lr = LogisticRegression(class_weight='balanced', max_iter=1000)
lr.fit(X_train, y_train)

lr_basic_train = lr.predict(X_train)

lr_basic_preds = lr.predict(X_test)

# Train Metrics
lr_basic_train_accuracy = metrics.accuracy_score(y_train, lr_basic_train)
lr_basic_train_precision = metrics.precision_score(y_train, lr_basic_train)
lr_basic_train_recall = metrics.recall_score(y_train, lr_basic_train)
lr_basic_train_f1 = metrics.f1_score(y_train, lr_basic_train)


# Test Metrics
lr_basic_accuracy = metrics.accuracy_score(y_test, lr_basic_preds)
lr_basic_precision = metrics.precision_score(y_test, lr_basic_preds)
lr_basic_recall = metrics.recall_score(y_test, lr_basic_preds)
lr_basic_f1 = metrics.f1_score(y_test, lr_basic_preds)



performance.append({'model': 'Basic Logistic Regression', '# Features': df_features.shape[1], 'Train Accuracy': lr_basic_train_accuracy,
                    'Test Accuracy': lr_basic_accuracy, 'Train Recall': lr_basic_train_recall,
                    'Test Recall': lr_basic_recall, 'Train Precision': lr_basic_train_precision,
                    'Test Precision': lr_basic_precision, 'Train F1': lr_basic_train_f1,
                    'Test F1': lr_basic_f1})

In [273]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.73308


In [174]:
# # Now, using a standard scaler instead of Min-Max (although, Min-Max is recommended for classification problems)
# # Instatiate the scaler
# scaler = StandardScaler()  

# # Fit the scaler to our training feature set
# scaler.fit(X_train)

# # Using the fitted scaler, transform the training feature set to values between 0 and 1
# X_train = scaler.transform(X_train)  

# # Transform the testing set as well to ensure consistency in predictions across feature sets
# X_test = scaler.transform(X_test) 

In [175]:
# # Fitting our basic logistic regression model.

# lr = LogisticRegression(class_weight='balanced', max_iter=1000)
# lr.fit(X_train, y_train)

# lr_basic_preds = lr.predict(X_test)

# lr_basic_f1 = metrics.f1_score(y_test, lr_basic_preds)

# print(lr_basic_f1)

In [176]:
# # Min-Max scaler performed better, so going to use that moving forward
# # Normalize our data using min-max scaler
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

# # Instatiate the scaler
# scaler = MinMaxScaler()  

# # Fit the scaler to our training feature set
# scaler.fit(X_train)

# # Using the fitted scaler, transform the training feature set to values between 0 and 1
# X_train = scaler.transform(X_train)  

# # Transform the testing set as well to ensure consistency in predictions across feature sets
# X_test = scaler.transform(X_test) 

In [177]:
# Applying gridsearchCV to model
from sklearn.model_selection import GridSearchCV

param_grid = { 
     "class_weight":['balanced'],
     "penalty": ['l1', 'l2'],
     "C":[0.001,0.01,0.1,1,10],
     "max_iter": [1000]
}

grid_log = GridSearchCV(LogisticRegression(), param_grid, cv=50, scoring='f1', verbose=1, n_jobs=-1)

grid_log.fit(X_train, y_train)

Fitting 50 folds for each of 10 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 260 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.3min finished


GridSearchCV(cv=50, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'class_weight': ['balanced'], 'max_iter': [1000],
                         'penalty': ['l1', 'l2']},
             scoring='f1', verbose=1)

In [178]:
print(grid_log.best_estimator_)
print(grid_log.best_params_)
print(grid_log.best_score_)

LogisticRegression(C=0.1, class_weight='balanced', max_iter=1000)
{'C': 0.1, 'class_weight': 'balanced', 'max_iter': 1000, 'penalty': 'l2'}
0.7226465527112268


In [274]:
# Predictions for training set
log_grid_preds_train = grid_log.best_estimator_.predict(X_train)

# Predictions for testing set
log_grid_preds = grid_log.best_estimator_.predict(X_test)

# Training set metrics
log_grid_train_accuracy = metrics.accuracy_score(y_train, log_grid_preds_train)
log_grid_train_precision = metrics.precision_score(y_train, log_grid_preds_train)
log_grid_train_recall = metrics.recall_score(y_train, log_grid_preds_train)
log_grid_train_f1 = metrics.f1_score(y_train, log_grid_preds_train)

# Testing set metrics
log_grid_accuracy = metrics.accuracy_score(y_test, log_grid_preds)
log_grid_precision = metrics.precision_score(y_test, log_grid_preds)
log_grid_recall = metrics.recall_score(y_test, log_grid_preds)
log_grid_f1 = metrics.f1_score(y_test, log_grid_preds)

performance.append({'model': 'GridSearchCV Logistic Regression', '# Features': df_features.shape[1],'Train Accuracy': log_grid_train_accuracy,
                    'Test Accuracy': log_grid_accuracy, 'Train Recall': log_grid_train_recall, 
                    'Test Recall': lr_basic_recall, 'Train Precision': log_grid_train_precision,
                    'Test Precision': log_grid_precision, 'Train F1': log_grid_train_f1,
                    'Test F1': log_grid_f1})

In [275]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163


## Feature Selection
### Significant Features Only
Before creating any additional models, let's see how our models perform with a feature set containing only significant features.

In [181]:
df_keep = pd.read_pickle(r'significant_features_df.pickle')

In [182]:
df_features_selected = df_features.loc[:,list(df_keep.index)]

In [183]:
# Conduct Train Test split on new feature set
X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(df_features_selected, df_target, 
                                                                    random_state=1, test_size=0.2)

In [184]:
# Normalize our data using min-max scaler

# Fit the scaler to our training feature set
scaler.fit(X_train_sel)

# Using the fitted scaler, transform the training feature set to values between 0 and 1
X_train_sel = scaler.transform(X_train_sel)  

# Transform the testing set as well to ensure consistency in predictions across feature sets
X_test_sel = scaler.transform(X_test_sel) 

In [276]:
# Fitting our basic logistic regression model.

lr = LogisticRegression(class_weight='balanced', max_iter=1000)
lr.fit(X_train_sel, y_train_sel)

lr_basic_train_sel = lr.predict(X_train_sel)

lr_basic_preds_sel = lr.predict(X_test_sel)

# Train Metrics
lr_basic_train_accuracy_sel = metrics.accuracy_score(y_train_sel, lr_basic_train_sel)
lr_basic_train_precision_sel = metrics.precision_score(y_train_sel, lr_basic_train_sel)
lr_basic_train_recall_sel = metrics.recall_score(y_train_sel, lr_basic_train_sel)
lr_basic_train_f1_sel = metrics.f1_score(y_train_sel, lr_basic_train_sel)


# Test Metrics
lr_basic_accuracy_sel = metrics.accuracy_score(y_test_sel, lr_basic_preds_sel)
lr_basic_precision_sel = metrics.precision_score(y_test_sel, lr_basic_preds_sel)
lr_basic_recall_sel = metrics.recall_score(y_test_sel, lr_basic_preds_sel)
lr_basic_f1_sel = metrics.f1_score(y_test_sel, lr_basic_preds_sel)


performance.append({'model': 'Basic Logistic Regression Selected', '# Features': df_features_selected.shape[1], 'Train Accuracy': lr_basic_train_accuracy_sel,
                    'Test Accuracy': lr_basic_accuracy_sel, 'Train Recall': lr_basic_train_recall_sel,
                    'Test Recall': lr_basic_recall_sel, 'Train Precision': lr_basic_train_precision_sel,
                    'Test Precision': lr_basic_precision_sel, 'Train F1': lr_basic_train_f1_sel,
                    'Test F1': lr_basic_f1_sel})

In [277]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824


In [188]:
# The model performed about as well as the previous models with fewer features. As such, I'll move forward with
# the feature set that has fewer features.

In [189]:
df_keep

,0,1,test_stat,p-value
pct_20,0.003612,0.001337,1.961900,0.049800
pct_109,0.021568,0.015709,1.986800,0.046900
descr_CRIMINAL POSSESSION OF FORGED INSTRUMENT,0.001912,0.004011,-2.035400,0.041800
pct_94,0.006694,0.003342,2.087700,0.036800
pct_33,0.004462,0.007687,-2.128200,0.033300
race_OTHER,0.012431,0.007687,2.139700,0.032400
pct_9,0.007544,0.011698,-2.150500,0.031500
city_MANHATTAN,0.198258,0.217246,-2.250000,0.024400
month_October,0.067042,0.079880,-2.395200,0.016600
sex_Other,0.005525,0.002005,2.458100,0.014000


In [190]:
df_features_selected.shape

(12404, 99)

### Recursive Feature Elimination with Smaller Feature Set
The smaller feature set did perform just about as well as previous models, although, the number of features is still fairly high for interpretability. As such, the number of features will be trimmed again using RFECV.

In [191]:
from sklearn.feature_selection import RFECV

selector = RFECV(LogisticRegression(class_weight='balanced', max_iter=1000), step=1, cv=10)
selector.fit(X_train_sel, y_train_sel)

RFECV(cv=10,
      estimator=LogisticRegression(class_weight='balanced', max_iter=1000))

In [192]:
print(selector.estimator_)
print(selector.n_features_)
print(selector.ranking_)
print(selector.support_)

LogisticRegression(class_weight='balanced', max_iter=1000)
15
[38 75 11 54 55 45 49 50 59 23 66 32 71 53 70 42 39  1 37  8 74 57 15 31
  1 79 52 65 13 46 51  1 25 22 17 29 63 67  5 81 14 26 34 68  4 28  1 72
  6 78  2  1  1  1 44  3  1  9 35 62 20  1 84  7  1  1 82 36 76 58 80 27
 43  1  1 77 24 16 33 60 12 48 40 69 73 21 19 10 47 30 83  1 64  1 61 18
 85 56 41]
[False False False False False False False False False False False False
 False False False False False  True False False False False False False
  True False False False False False False  True False False False False
 False False False False False False False False False False  True False
 False False False  True  True  True False False  True False False False
 False  True False False  True  True False False False False False False
 False  True  True False False False False False False False False False
 False False False False False False False  True False  True False False
 False False False]


In [279]:
# Using the fitted scaler, transform the training feature set to values between 0 and 1
X_train_rfe = selector.transform(X_train_sel)  

# Transform the testing set as well to ensure consistency in predictions across feature sets
X_test_rfe = selector.transform(X_test_sel) 

# Predict on the training set
rfe_train_preds = selector.estimator_.predict(X_train_rfe)

# Predict on the testing set
rfe_test_preds = selector.estimator_.predict(X_test_rfe)

# Train Metrics
basic_lr_sel_rfe_accuracy = metrics.accuracy_score(y_train_sel, rfe_train_preds)
basic_lr_sel_rfe_precision = metrics.precision_score(y_train_sel, rfe_train_preds)
basic_lr_sel_rfe_recall = metrics.recall_score(y_train_sel, rfe_train_preds)
basic_lr_sel_rfe_f1 = metrics.f1_score(y_train_sel, rfe_train_preds)


# Test Metrics
basic_lr_sel_rfe_test_accuracy = metrics.accuracy_score(y_test_sel, rfe_test_preds)
basic_lr_sel_rfe_test_precision = metrics.precision_score(y_test_sel, rfe_test_preds)
basic_lr_sel_rfe_test_recall = metrics.recall_score(y_test_sel, rfe_test_preds)
basic_lr_sel_rfe_test_f1 = metrics.f1_score(y_test_sel, rfe_test_preds)

performance.append({'model': 'Basic LogReg w/ Selected Features & RFECV', '# Features': X_train_rfe.shape[1], 'Train Accuracy': basic_lr_sel_rfe_accuracy,
                    'Test Accuracy': basic_lr_sel_rfe_test_accuracy, 'Train Recall': basic_lr_sel_rfe_recall,
                    'Test Recall': basic_lr_sel_rfe_test_recall, 'Train Precision': basic_lr_sel_rfe_precision,
                    'Test Precision': basic_lr_sel_rfe_test_precision, 'Train F1': basic_lr_sel_rfe_f1,
                    'Test F1': basic_lr_sel_rfe_test_f1})

In [280]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776


In [195]:
print(selector.support_)

[False False False False False False False False False False False False
 False False False False False  True False False False False False False
  True False False False False False False  True False False False False
 False False False False False False False False False False  True False
 False False False  True  True  True False False  True False False False
 False  True False False  True  True False False False False False False
 False  True  True False False False False False False False False False
 False False False False False False False  True False  True False False
 False False False]


In [221]:
# Creating a list of selected feature indices. In the list above, True denotes a feature that was selected to 
# bw included in the model. I'll add each of these indices to a list to later parse out ans explore. 
index_sel = []
for i in range(len(selector.support_)):
    if selector.support_[i]==True:
        index_sel.append(i)
    else:
        continue

rfecv_feats = list(df_features.iloc[:, index_sel].columns)

After recursive feature elimination, the top 15 features that remeained in the model were:
- **pf_drwep**: Police force used - weapon drawn
- **ac_inves**: Additional Circumstances - ongoing investivation (i.e., the person was potentially a suspect in an ongoing investigation)
- **cs_casng**: Reason for stop - casing a victim or location
- **ac_time**: Additional Circumstances - time of day fits additional crime incidence
- **sb_outln**: Basis of search - outline of weapon
- **sb_admis**: Basis of search - admission by suspect
- **sb_other**: Basis of search - other
- **age**: suspect's age was a strong predictor of arrest. Younger individuals were more likely to be stopped and arrested.
- **pct_7**, **pct_13**, **pct_14**, **pct_25**, **pct_26**, **pct_52**, **pct_61**: The precint nearest to the incident. These precincts must have higher arrest rates as a proportion of SQF incidents.

Let's take a similar approach to modeling with Decision Trees and see which features come out on top. We can then conduct some further interactions with the best performing features to observe performance. 

## Decision Tree Models

In [281]:
# Create a baseline decision tree model using the original feature set

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

# Instantiate the decision tree classifier using all default parameters and class weight balanced due to class 
# imbalance
clf = DecisionTreeClassifier(class_weight='balanced', random_state=1)

# Fit the model to our training data
clf_basic = clf.fit(X_train, y_train)

# Make predictions for y_values in the training set
dtc_basic_train_preds = clf_basic.predict(X_train)

# Make predictions for y values in the testing dataset
dtc_basic_test_preds = clf_basic.predict(X_test)

# Train Metrics
dtc_basic_train_accuracy = metrics.accuracy_score(y_train, dtc_basic_train_preds)
dtc_basic_train_precision = metrics.precision_score(y_train, dtc_basic_train_preds)
dtc_basic_train_recall = metrics.recall_score(y_train, dtc_basic_train_preds)
dtc_basic_train_f1 = metrics.f1_score(y_train, dtc_basic_train_preds)


# Test Metrics
dtc_basic_accuracy = metrics.accuracy_score(y_test, dtc_basic_test_preds)
dtc_basic_precision = metrics.precision_score(y_test, dtc_basic_test_preds)
dtc_basic_recall = metrics.recall_score(y_test, dtc_basic_test_preds)
dtc_basic_f1 = metrics.f1_score(y_test, dtc_basic_test_preds)


performance.append({'model': 'Basic Decision Tree Model', '# Features': X_train.shape[1], 'Train Accuracy': dtc_basic_train_accuracy,
                    'Test Accuracy': dtc_basic_accuracy, 'Train Recall': dtc_basic_train_recall,
                    'Test Recall': dtc_basic_recall, 'Train Precision': dtc_basic_train_precision,
                    'Test Precision': dtc_basic_precision, 'Train F1': dtc_basic_train_f1,
                    'Test F1': dtc_basic_f1})

In [282]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795


In [199]:
# It seems the Decision Tree Severly overfit to the training data. A GridSearch decision tree will be tested to 
# see if this improves performance.

dt_params = {'max_depth': [2, 3, 5, 10, 15, 20],
              'criterion': ['gini', 'entropy'],
              'min_samples_leaf': [2,5,10],
              'min_samples_split': [5,7,10]}

In [200]:
# Create the instance
grid_dt_basic = GridSearchCV(clf, dt_params, cv=10, scoring='f1', verbose=1, n_jobs=-1)

# Fit the decision tree classifier to the training data
grid_dt_basic.fit(X_train, y_train)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   40.0s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:   56.0s finished


GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 15, 20],
                         'min_samples_leaf': [2, 5, 10],
                         'min_samples_split': [5, 7, 10]},
             scoring='f1', verbose=1)

In [201]:
# Show the parameters for the best model and the f1 score for the best model
print('Best Score: %f' % (grid_dt_basic.best_score_))
print('Best Params:', grid_dt_basic.best_params_)

Best Score: 0.725267
Best Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5}


In [283]:
# Make predictions for y values in the training set
dtc_grid_train_preds = grid_dt_basic.best_estimator_.predict(X_train)

# Make predictions for y values in the testing dataset
dtc_grid_test_preds = grid_dt_basic.best_estimator_.predict(X_test)

# Train Metrics
dtc_grid_train_accuracy = metrics.accuracy_score(y_train, dtc_grid_train_preds)
dtc_grid_train_precision = metrics.precision_score(y_train, dtc_grid_train_preds)
dtc_grid_train_recall = metrics.recall_score(y_train, dtc_grid_train_preds)
dtc_grid_train_f1 = metrics.f1_score(y_train, dtc_grid_train_preds)


# Test Metrics
dtc_grid_accuracy = metrics.accuracy_score(y_test, dtc_grid_test_preds)
dtc_grid_precision = metrics.precision_score(y_test, dtc_grid_test_preds)
dtc_grid_recall = metrics.recall_score(y_test, dtc_grid_test_preds)
dtc_grid_f1 = metrics.f1_score(y_test, dtc_grid_test_preds)


performance.append({'model': 'GridSearchCV Decision Tree Model', '# Features': X_train.shape[1], 'Train Accuracy': dtc_grid_train_accuracy,
                    'Test Accuracy': dtc_grid_accuracy, 'Train Recall': dtc_grid_train_recall,
                    'Test Recall': dtc_grid_recall, 'Train Precision': dtc_grid_train_precision,
                    'Test Precision': dtc_grid_precision, 'Train F1': dtc_grid_train_f1,
                    'Test F1': dtc_grid_f1})

In [284]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952


## Feature Selection
### Significant Features Only

In [285]:
# Create a baseline decision tree model using the significant feature set

# Fit the model to our training data
clf_basic_sel = clf.fit(X_train_sel, y_train_sel)

# Make predictions for y_values in the training set
dtc_basic_train_preds_sel = clf_basic_sel.predict(X_train_sel)

# Make predictions for y values in the testing dataset
dtc_basic_test_preds_sel = clf_basic_sel.predict(X_test_sel)

# Train Metrics
dtc_basic_train_accuracy_sel = metrics.accuracy_score(y_train_sel, dtc_basic_train_preds_sel)
dtc_basic_train_precision_sel = metrics.precision_score(y_train_sel, dtc_basic_train_preds_sel)
dtc_basic_train_recall_sel = metrics.recall_score(y_train_sel, dtc_basic_train_preds_sel)
dtc_basic_train_f1_sel = metrics.f1_score(y_train_sel, dtc_basic_train_preds_sel)

# Test Metrics
dtc_basic_accuracy_sel = metrics.accuracy_score(y_test_sel, dtc_basic_test_preds_sel)
dtc_basic_precision_sel = metrics.precision_score(y_test_sel, dtc_basic_test_preds_sel)
dtc_basic_recall_sel = metrics.recall_score(y_test_sel, dtc_basic_test_preds_sel)
dtc_basic_f1_sel = metrics.f1_score(y_test_sel, dtc_basic_test_preds_sel)

performance.append({'model': 'Basic Decision Tree Selected Features', '# Features': X_train_sel.shape[1], 'Train Accuracy': dtc_basic_train_accuracy_sel,
                    'Test Accuracy': dtc_basic_accuracy_sel, 'Train Recall': dtc_basic_train_recall_sel,
                    'Test Recall': dtc_basic_recall_sel, 'Train Precision': dtc_basic_train_precision_sel,
                    'Test Precision': dtc_basic_precision_sel, 'Train F1': dtc_basic_train_f1_sel,
                    'Test F1': dtc_basic_f1_sel})

In [286]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952
6,Basic Decision Tree Selected Features,99,1.000000,0.836356,1.000000,0.676033,1.000000,0.660743,1.000000,0.668301


### GridsearchCV Decision Tree with Smaller Feature Set

In [209]:
# Create the instance
grid_dt_sel = GridSearchCV(clf, dt_params, cv=10, scoring='f1', verbose=1, n_jobs=-1)

# Fit the decision tree classifier to the training data
grid_dt_sel.fit(X_train_sel, y_train_sel)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:   19.1s finished


GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 15, 20],
                         'min_samples_leaf': [2, 5, 10],
                         'min_samples_split': [5, 7, 10]},
             scoring='f1', verbose=1)

In [210]:
# Show the parameters for the best model and the f1 score for the best model
print('Best Score: %f' % (grid_dt_sel.best_score_))
print('Best Params:', grid_dt_sel.best_params_)

Best Score: 0.716587
Best Params: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 7}


In [287]:
# Make predictions for y values in the training set
dtc_grid_train_preds_sel = grid_dt_sel.best_estimator_.predict(X_train_sel)

# Make predictions for y values in the testing dataset
dtc_grid_test_preds_sel = grid_dt_sel.best_estimator_.predict(X_test_sel)

# Train Metrics
dtc_grid_train_accuracy_sel = metrics.accuracy_score(y_train_sel, dtc_grid_train_preds_sel)
dtc_grid_train_precision_sel = metrics.precision_score(y_train_sel, dtc_grid_train_preds_sel)
dtc_grid_train_recall_sel = metrics.recall_score(y_train_sel, dtc_grid_train_preds_sel)
dtc_grid_train_f1_sel = metrics.f1_score(y_train_sel, dtc_grid_train_preds_sel)

# Test Metrics
dtc_grid_accuracy_sel = metrics.accuracy_score(y_test_sel, dtc_grid_test_preds_sel)
dtc_grid_precision_sel = metrics.precision_score(y_test_sel, dtc_grid_test_preds_sel)
dtc_grid_recall_sel = metrics.recall_score(y_test_sel, dtc_grid_test_preds_sel)
dtc_grid_f1_sel = metrics.f1_score(y_test_sel, dtc_grid_test_preds_sel)


performance.append({'model': 'GridSearchCV Decision Tree Model w/ Selected Features', '# Features': X_train_sel.shape[1], 'Train Accuracy': dtc_grid_train_accuracy_sel,
                    'Test Accuracy': dtc_grid_accuracy_sel, 'Train Recall': dtc_grid_train_recall_sel,
                    'Test Recall': dtc_grid_recall_sel, 'Train Precision': dtc_grid_train_precision_sel,
                    'Test Precision': dtc_grid_precision_sel, 'Train F1': dtc_grid_train_f1_sel,
                    'Test F1': dtc_grid_f1_sel})

In [288]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952
6,Basic Decision Tree Selected Features,99,1.000000,0.836356,1.000000,0.676033,1.000000,0.660743,1.000000,0.668301
7,GridSearchCV Decision Tree Model w/ Selected F...,99,0.894891,0.868198,0.775450,0.735537,0.784987,0.727124,0.780190,0.731306


In [253]:
list_of_tuples = list(zip(list(df_features_selected.columns), list(grid_dt_sel.best_estimator_.feature_importances_)))
grid_feat_perf = pd.DataFrame(list_of_tuples, columns = ['features', 'feature_importance'])

In [256]:
grid_feat_perf = grid_feat_perf.sort_values(by='feature_importance', ascending=False)

In [257]:
grid_feat_perf.reset_index(inplace=True)

In [258]:
grid_feat_perf

,level_0,index,features,feature_importance
0,64,64,searched,4.852550e-01
1,53,53,contrabn,1.140421e-01
2,52,52,descr_CRIMINAL TRESPASS,7.484658e-02
3,55,55,pf_hcuff,3.385230e-02
4,91,91,sb_hdobj,2.571661e-02
5,61,61,knifcuti,2.313153e-02
6,97,97,age,2.048928e-02
7,56,56,pistol,1.823258e-02
8,77,77,pct_106,1.480994e-02
9,98,98,weight,1.352497e-02


Re-running **GridSearchCV Decision Tree Model** w/ only those features in the above list that have a value greater than 0.

In [259]:
non_zero_feat_importance = list(grid_feat_perf.features[:66])

In [260]:
df_features_temp = df_features.loc[:,non_zero_feat_importance]

In [261]:
# Conduct Train Test split on new feature set
X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(df_features_temp, df_target, 
                                                                    random_state=1, test_size=0.2)

In [262]:
# Create the instance
grid_dt_ftimp = GridSearchCV(clf, dt_params, cv=10, scoring='f1', verbose=1, n_jobs=-1)

# Fit the decision tree classifier to the training data
grid_dt_ftimp.fit(X_train_temp, y_train_temp)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 440 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:   10.4s finished


GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 15, 20],
                         'min_samples_leaf': [2, 5, 10],
                         'min_samples_split': [5, 7, 10]},
             scoring='f1', verbose=1)

In [263]:
# Show the parameters for the best model and the f1 score for the best model
print('Best Score: %f' % (grid_dt_ftimp.best_score_))
print('Best Params:', grid_dt_ftimp.best_params_)

Best Score: 0.719128
Best Params: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5}


In [289]:
# Make predictions for y values in the training set
dtc_grid_train_preds_temp = grid_dt_ftimp.best_estimator_.predict(X_train_temp)

# Make predictions for y values in the testing dataset
dtc_grid_test_preds_temp = grid_dt_ftimp.best_estimator_.predict(X_test_temp)

# Train Metrics
dtc_grid_train_accuracy_temp = metrics.accuracy_score(y_train_temp, dtc_grid_train_preds_temp)
dtc_grid_train_precision_temp = metrics.precision_score(y_train_temp, dtc_grid_train_preds_temp)
dtc_grid_train_recall_temp = metrics.recall_score(y_train_temp, dtc_grid_train_preds_temp)
dtc_grid_train_f1_temp = metrics.f1_score(y_train_temp, dtc_grid_train_preds_temp)

# Test Metrics
dtc_grid_accuracy_temp = metrics.accuracy_score(y_test_temp, dtc_grid_test_preds_temp)
dtc_grid_precision_temp = metrics.precision_score(y_test_temp, dtc_grid_test_preds_temp)
dtc_grid_recall_temp = metrics.recall_score(y_test_temp, dtc_grid_test_preds_temp)
dtc_grid_f1_temp = metrics.f1_score(y_test_temp, dtc_grid_test_preds_temp)


performance.append({'model': 'GridSearchCV Decision Tree Model w/ Feature Importance', '# Features': X_train_temp.shape[1], 'Train Accuracy': dtc_grid_train_accuracy_temp,
                    'Test Accuracy': dtc_grid_accuracy_temp, 'Train Recall': dtc_grid_train_recall_temp,
                    'Test Recall': dtc_grid_recall_temp, 'Train Precision': dtc_grid_train_precision_temp,
                    'Test Precision': dtc_grid_precision_temp, 'Train F1': dtc_grid_train_f1_temp,
                    'Test F1': dtc_grid_f1_temp})

In [290]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952
6,Basic Decision Tree Selected Features,99,1.000000,0.836356,1.000000,0.676033,1.000000,0.660743,1.000000,0.668301
7,GridSearchCV Decision Tree Model w/ Selected F...,99,0.894891,0.868198,0.775450,0.735537,0.784987,0.727124,0.780190,0.731306
8,GridSearchCV Decision Tree Model w/ Feature Im...,66,0.895697,0.869811,0.775869,0.737190,0.787415,0.731148,0.781599,0.734156


This model slightly improved performance with respect to the model that included more features, so let's narrow down these features even more. The above model ran 65 features that were deemed important by the model, but let's see how the model performs when selecting the top 15 features and combining these with the most imporatnce features from RFECV for our logistic regression model. 

In [222]:
grid_cv_top15 = list(grid_feat_perf.features[0:15])

In [223]:
grid_cv_top15

['searched',
 'contrabn',
 'descr_CRIMINAL TRESPASS',
 'pf_hcuff',
 'sb_hdobj',
 'knifcuti',
 'age',
 'pistol',
 'pct_106',
 'weight',
 'descr_THEFT OF SERVICES',
 'descr_CPW',
 'othrweap',
 'trhsloc_Neither',
 'descr_PETIT LARCENY']

After a running GridSearch for our decision tree model, the above features have come out and the top 15 most important features in predictions. These 15 features are not the only features included in the decision tree model, however, they were chosen fo rthe sake of comparing feature importance to the RFECV logistic regression model and selecting features consistent in both models for interactions. 
- **searched**: The fact that an individual was searched
- **contrbn**: Some form of contaband was found on the suspect (this could include a weapon, drugs, etc).
- **descr_CRIMINAL TRESPASS**: Individuals found trespassing were more likely to be arrested.
- **pf_hcuff**: The fact that an individual was placed in handcuffs makes it more likely that they would be arrested. It should be noted there were also individuals who were placed in handcuffs who were not ultimately arrested.
- **sb_hdobj**: Basis of search - hard object (likely found during frisk).
- **knifcuti**: Individuals with knives found in their possession were more likely to be arrested.
- **age**: Again, age plays a large role as mostly younger people are arrested.
- **pistol**: pistol found in the suspect's possession is more likely to result in an arrest.
- **pct_106**: This precinct likely arrests individuals at higher rates than others likely to dur to location and crime rate in that area. 
- **weight**: An individual's weight is a likely contributor to likelihood of arrest. This could be due to the correlation between weight and age. 
- **descr_THEFT OF SERVICES**: When an individual obtains services without compensating the provider.
- **descr_CPW**: Criminal possession of a weapon
- **othrweap**: Suspect is in possession of another weapon other than those listed. 
- **trhsloc**: Location of arrest (options are Transit, Public Housing, Neither). Neither is more likely to result in arrest (likely because most crimes are committed in the street). 
- **descr_PETIT LARCENY**: theft of property less than $1,000.


These features will be combined with the best features from logistic regression RFECV to create additional models and feature interactions. 

## Finalizing Feature Set

In [249]:
# Combining these features with those previosuly shown to be most important through recursive feature elimination,
# we get the following list of features. I've used the set function to remove duplicates. The only duplicate
# feature was age meaning that this was the most robust feature in the dataset. 
duplicates = []
for each in rfecv_feats:
    if each in grid_cv_top15:
        duplicates.append(each)
    else:
        continue
print(duplicates)

important_features = list(set(rfecv_feats + grid_cv_top15))
print(important_features)

['age']
['descr_CRIMINAL TRESPASS', 'ac_time', 'pct_7', 'weight', 'pct_61', 'cs_casng', 'searched', 'sb_other', 'descr_THEFT OF SERVICES', 'ac_inves', 'pct_14', 'pct_26', 'pf_drwep', 'contrabn', 'sb_admis', 'trhsloc_Neither', 'descr_PETIT LARCENY', 'pistol', 'pct_13', 'pct_52', 'sb_outln', 'knifcuti', 'pct_106', 'othrweap', 'pf_hcuff', 'age', 'pct_25', 'descr_CPW', 'sb_hdobj']


So far, the best performing models with fewer features (for interpretability) are Basic LogReg w/ Selected Features & RFECV and GridSearchCV Decision Tree Model w/ Feature Importance. I'll work with these two when paring down the feature set and creating interaction features.

I'll first start by running each of these models with just the final feature set.

In [292]:
final_features = df_features.loc[:,important_features]

In [293]:
# Conduct Train Test split on new feature set
X_train_fin, X_test_fin, y_train_fin, y_test_fin = train_test_split(final_features, df_target, 
                                                                    random_state=1, test_size=0.2)

### Basic LogReg w/ Final Features & RFECV

In [294]:
selector = RFECV(LogisticRegression(class_weight='balanced', max_iter=1000), step=1, cv=10)
selector.fit(X_train_fin, y_train_fin)

RFECV(cv=10,
      estimator=LogisticRegression(class_weight='balanced', max_iter=1000))

In [295]:
print(selector.estimator_)
print(selector.n_features_)
print(selector.ranking_)
print(selector.support_)

LogisticRegression(class_weight='balanced', max_iter=1000)
14
[ 1 13 11 15  6  7  1  1  1 12  9  3 10  1  2  4  1  1  1  5  8  1  1  1
  1 16 14  1  1]
[ True False False False False False  True  True  True False False False
 False  True False False  True  True  True False False  True  True  True
  True False False  True  True]


In [297]:
# Creating a list of selected feature indices. In the list above, True denotes a feature that was selected to 
# bw included in the model. I'll add each of these indices to a list to later parse out and explore. 
index_sel_fin = []
for i in range(len(selector.support_)):
    if selector.support_[i]==True:
        index_sel_fin.append(i)
    else:
        continue

rfecv_feats_fin = list(final_features.iloc[:, index_sel_fin].columns)
rfecv_feats_fin

['descr_CRIMINAL TRESPASS',
 'searched',
 'sb_other',
 'descr_THEFT OF SERVICES',
 'contrabn',
 'descr_PETIT LARCENY',
 'pistol',
 'pct_13',
 'knifcuti',
 'pct_106',
 'othrweap',
 'pf_hcuff',
 'descr_CPW',
 'sb_hdobj']

In [298]:
# Using the fitted scaler, transform the training feature set to values between 0 and 1
X_train_rfe_fin = selector.transform(X_train_fin)  

# Transform the testing set as well to ensure consistency in predictions across feature sets
X_test_rfe_fin = selector.transform(X_test_fin) 

# Predict on the training set
fin_rfe_train_preds = selector.estimator_.predict(X_train_rfe_fin)

# Predict on the testing set
fin_rfe_test_preds = selector.estimator_.predict(X_test_rfe_fin)

# Train Metrics
basic_lr_fin_rfe_accuracy = metrics.accuracy_score(y_train_fin, fin_rfe_train_preds)
basic_lr_fin_rfe_precision = metrics.precision_score(y_train_fin, fin_rfe_train_preds)
basic_lr_fin_rfe_recall = metrics.recall_score(y_train_fin, fin_rfe_train_preds)
basic_lr_fin_rfe_f1 = metrics.f1_score(y_train_fin, fin_rfe_train_preds)


# Test Metrics
basic_lr_fin_rfe_test_accuracy = metrics.accuracy_score(y_test_fin, fin_rfe_test_preds)
basic_lr_fin_rfe_test_precision = metrics.precision_score(y_test_fin, fin_rfe_test_preds)
basic_lr_fin_rfe_test_recall = metrics.recall_score(y_test_fin, fin_rfe_test_preds)
basic_lr_fin_rfe_test_f1 = metrics.f1_score(y_test_fin, fin_rfe_test_preds)

performance.append({'model': 'Basic LogReg w/ Final Features & RFECV', '# Features': X_train_fin.shape[1], 'Train Accuracy': basic_lr_fin_rfe_accuracy,
                    'Test Accuracy': basic_lr_fin_rfe_test_accuracy, 'Train Recall': basic_lr_fin_rfe_recall,
                    'Test Recall': basic_lr_fin_rfe_test_recall, 'Train Precision': basic_lr_fin_rfe_precision,
                    'Test Precision': basic_lr_fin_rfe_test_precision, 'Train F1': basic_lr_fin_rfe_f1,
                    'Test F1': basic_lr_fin_rfe_test_f1})

In [299]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952
6,Basic Decision Tree Selected Features,99,1.000000,0.836356,1.000000,0.676033,1.000000,0.660743,1.000000,0.668301
7,GridSearchCV Decision Tree Model w/ Selected F...,99,0.894891,0.868198,0.775450,0.735537,0.784987,0.727124,0.780190,0.731306
8,GridSearchCV Decision Tree Model w/ Feature Im...,66,0.895697,0.869811,0.775869,0.737190,0.787415,0.731148,0.781599,0.734156
9,Basic LogReg w/ Final Features & RFECV,29,0.858007,0.851270,0.772937,0.780165,0.680310,0.666667,0.723671,0.718964


This model performed slightly worse than original Basic LogReg model with RFECV (3). I'll try this feature set now with a GridSearchCV Decision Tree Model.

### GridSearchCV Decision Tree Model with Final Features

In [300]:
# Create the instance
grid_dt_fin = GridSearchCV(clf, dt_params, cv=10, scoring='f1', verbose=1, n_jobs=-1)

# Fit the decision tree classifier to the training data
grid_dt_fin.fit(X_train_fin, y_train_fin)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 404 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:    7.5s finished


GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 15, 20],
                         'min_samples_leaf': [2, 5, 10],
                         'min_samples_split': [5, 7, 10]},
             scoring='f1', verbose=1)

In [301]:
# Make predictions for y values in the training set
dtc_grid_train_preds_fin = grid_dt_fin.best_estimator_.predict(X_train_fin)

# Make predictions for y values in the testing dataset
dtc_grid_test_preds_fin = grid_dt_fin.best_estimator_.predict(X_test_fin)

# Train Metrics
dtc_grid_train_accuracy_fin = metrics.accuracy_score(y_train_fin, dtc_grid_train_preds_fin)
dtc_grid_train_precision_fin = metrics.precision_score(y_train_fin, dtc_grid_train_preds_fin)
dtc_grid_train_recall_fin = metrics.recall_score(y_train_fin, dtc_grid_train_preds_fin)
dtc_grid_train_f1_fin = metrics.f1_score(y_train_fin, dtc_grid_train_preds_fin)

# Test Metrics
dtc_grid_accuracy_fin = metrics.accuracy_score(y_test_fin, dtc_grid_test_preds_fin)
dtc_grid_precision_fin = metrics.precision_score(y_test_fin, dtc_grid_test_preds_fin)
dtc_grid_recall_fin = metrics.recall_score(y_test_fin, dtc_grid_test_preds_fin)
dtc_grid_f1_fin = metrics.f1_score(y_test_fin, dtc_grid_test_preds_fin)


performance.append({'model': 'GridSearchCV Decision Tree Model w/ Final Features', '# Features': X_train_fin.shape[1], 'Train Accuracy': dtc_grid_train_accuracy_fin,
                    'Test Accuracy': dtc_grid_accuracy_fin, 'Train Recall': dtc_grid_train_recall_fin,
                    'Test Recall': dtc_grid_recall_fin, 'Train Precision': dtc_grid_train_precision_fin,
                    'Test Precision': dtc_grid_precision_fin, 'Train F1': dtc_grid_train_f1_fin,
                    'Test F1': dtc_grid_f1_fin})

In [302]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952
6,Basic Decision Tree Selected Features,99,1.000000,0.836356,1.000000,0.676033,1.000000,0.660743,1.000000,0.668301
7,GridSearchCV Decision Tree Model w/ Selected F...,99,0.894891,0.868198,0.775450,0.735537,0.784987,0.727124,0.780190,0.731306
8,GridSearchCV Decision Tree Model w/ Feature Im...,66,0.895697,0.869811,0.775869,0.737190,0.787415,0.731148,0.781599,0.734156
9,Basic LogReg w/ Final Features & RFECV,29,0.858007,0.851270,0.772937,0.780165,0.680310,0.666667,0.723671,0.718964


This model seems to be the winner so far with the highest F1 score and fit on both the training and test sets. 

In [310]:
# Reviewing feature importance
list_of_tuples = list(zip(list(final_features.columns), list(grid_dt_fin.best_estimator_.feature_importances_)))
fin_feat_perf = pd.DataFrame(list_of_tuples, columns = ['features', 'feature_importance'])

In [311]:
fin_feat_perf = fin_feat_perf.sort_values(by='feature_importance', ascending=False)

In [313]:
fin_feat_perf.reset_index(inplace=True)

In [314]:
fin_feat_perf

,index,features,feature_importance
0,6,searched,0.521073
1,13,contrabn,0.122460
2,0,descr_CRIMINAL TRESPASS,0.079757
3,24,pf_hcuff,0.039706
4,25,age,0.036310
5,28,sb_hdobj,0.027615
6,21,knifcuti,0.026604
7,3,weight,0.024788
8,17,pistol,0.019486
9,22,pct_106,0.017992


In [315]:
# Re-running the model removing non-important features
top_feat_importance = list(fin_feat_perf.features[:24])

In [316]:
final_features_temp = final_features.loc[:,top_feat_importance]

In [318]:
# Conduct Train Test split on new feature set
X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(final_features_temp, df_target, 
                                                                    random_state=1, test_size=0.2)

In [319]:
# Create the instance
fin_grid = GridSearchCV(clf, dt_params, cv=10, scoring='f1', verbose=1, n_jobs=-1)

# Fit the decision tree classifier to the training data
fin_grid.fit(X_train_temp, y_train_temp)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 440 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:    6.1s finished


GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 15, 20],
                         'min_samples_leaf': [2, 5, 10],
                         'min_samples_split': [5, 7, 10]},
             scoring='f1', verbose=1)

In [320]:
# Make predictions for y values in the training set
fin_dtgrid_train_preds = fin_grid.best_estimator_.predict(X_train_temp)

# Make predictions for y values in the testing dataset
fin_dtgrid_test_preds = fin_grid.best_estimator_.predict(X_test_temp)

# Train Metrics
fingrid_train_accuracy_temp = metrics.accuracy_score(y_train_temp, fin_dtgrid_train_preds)
fingrid_train_precision_temp = metrics.precision_score(y_train_temp, fin_dtgrid_train_preds)
fingrid_train_recall_temp = metrics.recall_score(y_train_temp, fin_dtgrid_train_preds)
fingrid_train_f1_temp = metrics.f1_score(y_train_temp, fin_dtgrid_train_preds)

# Test Metrics
fingrid_accuracy_temp = metrics.accuracy_score(y_test_temp, fin_dtgrid_test_preds)
fingrid_precision_temp = metrics.precision_score(y_test_temp, fin_dtgrid_test_preds)
fingrid_recall_temp = metrics.recall_score(y_test_temp, fin_dtgrid_test_preds)
fingrid_f1_temp = metrics.f1_score(y_test_temp, fin_dtgrid_test_preds)


performance.append({'model': 'Final GridSearchCV Decision Tree Model w/ Feature Importance', '# Features': X_train_temp.shape[1], 'Train Accuracy': fingrid_train_accuracy_temp,
                    'Test Accuracy': fingrid_accuracy_temp, 'Train Recall': fingrid_train_recall_temp,
                    'Test Recall': fingrid_recall_temp, 'Train Precision': fingrid_train_precision_temp,
                    'Test Precision': fingrid_precision_temp, 'Train F1': fingrid_train_f1_temp,
                    'Test F1': fingrid_f1_temp})

In [321]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952
6,Basic Decision Tree Selected Features,99,1.000000,0.836356,1.000000,0.676033,1.000000,0.660743,1.000000,0.668301
7,GridSearchCV Decision Tree Model w/ Selected F...,99,0.894891,0.868198,0.775450,0.735537,0.784987,0.727124,0.780190,0.731306
8,GridSearchCV Decision Tree Model w/ Feature Im...,66,0.895697,0.869811,0.775869,0.737190,0.787415,0.731148,0.781599,0.734156
9,Basic LogReg w/ Final Features & RFECV,29,0.858007,0.851270,0.772937,0.780165,0.680310,0.666667,0.723671,0.718964


This model performed just as well as the previous model with fewer features, so 11 is the winner as of now.

## Creating Poly Feature Set with Most Important Features

In [323]:
final_features_temp.head()

,searched,contrabn,descr_CRIMINAL TRESPASS,pf_hcuff,age,sb_hdobj,knifcuti,weight,pistol,pct_106,trhsloc_Neither,descr_CPW,othrweap,descr_PETIT LARCENY,descr_THEFT OF SERVICES,ac_time,cs_casng,sb_other,pf_drwep,ac_inves,sb_admis,pct_52,pct_61,pct_13
0,1,0,0,1,48,0,0,180,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0
1,1,0,0,0,20,0,0,150,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0
2,1,0,0,0,34,0,0,140,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,1,33,0,0,180,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,22,0,0,160,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [324]:
from sklearn.preprocessing import PolynomialFeatures

#Second degree polynomial
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(final_features)
poly2_columns = poly_2.get_feature_names(final_features.columns)
df_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)
df_poly2.head()

,descr_CRIMINAL TRESPASS,ac_time,pct_7,weight,pct_61,cs_casng,searched,sb_other,descr_THEFT OF SERVICES,ac_inves,pct_14,pct_26,pf_drwep,contrabn,sb_admis,trhsloc_Neither,descr_PETIT LARCENY,pistol,pct_13,pct_52,sb_outln,knifcuti,pct_106,othrweap,pf_hcuff,age,pct_25,descr_CPW,sb_hdobj,descr_CRIMINAL TRESPASS^2,descr_CRIMINAL TRESPASS ac_time,descr_CRIMINAL TRESPASS pct_7,descr_CRIMINAL TRESPASS weight,descr_CRIMINAL TRESPASS pct_61,descr_CRIMINAL TRESPASS cs_casng,descr_CRIMINAL TRESPASS searched,descr_CRIMINAL TRESPASS sb_other,descr_CRIMINAL TRESPASS descr_THEFT OF SERVICES,descr_CRIMINAL TRESPASS ac_inves,descr_CRIMINAL TRESPASS pct_14,descr_CRIMINAL TRESPASS pct_26,descr_CRIMINAL TRESPASS pf_drwep,descr_CRIMINAL TRESPASS contrabn,descr_CRIMINAL TRESPASS sb_admis,descr_CRIMINAL TRESPASS trhsloc_Neither,descr_CRIMINAL TRESPASS descr_PETIT LARCENY,descr_CRIMINAL TRESPASS pistol,descr_CRIMINAL TRESPASS pct_13,descr_CRIMINAL TRESPASS pct_52,descr_CRIMINAL TRESPASS sb_outln,descr_CRIMINAL TRESPASS knifcuti,descr_CRIMINAL TRESPASS pct_106,descr_CRIMINAL TRESPASS othrweap,descr_CRIMINAL TRESPASS pf_hcuff,descr_CRIMINAL TRESPASS age,descr_CRIMINAL TRESPASS pct_25,descr_CRIMINAL TRESPASS descr_CPW,descr_CRIMINAL TRESPASS sb_hdobj,ac_time^2,ac_time pct_7,ac_time weight,ac_time pct_61,ac_time cs_casng,ac_time searched,ac_time sb_other,ac_time descr_THEFT OF SERVICES,ac_time ac_inves,ac_time pct_14,ac_time pct_26,ac_time pf_drwep,ac_time contrabn,ac_time sb_admis,ac_time trhsloc_Neither,ac_time descr_PETIT LARCENY,ac_time pistol,ac_time pct_13,ac_time pct_52,ac_time sb_outln,ac_time knifcuti,ac_time pct_106,ac_time othrweap,ac_time pf_hcuff,ac_time age,ac_time pct_25,ac_time descr_CPW,ac_time sb_hdobj,pct_7^2,pct_7 weight,pct_7 pct_61,pct_7 cs_casng,pct_7 searched,pct_7 sb_other,pct_7 descr_THEFT OF SERVICES,pct_7 ac_inves,pct_7 pct_14,pct_7 pct_26,pct_7 pf_drwep,pct_7 contrabn,pct_7 sb_admis,pct_7 trhsloc_Neither,...,trhsloc_Neither sb_outln,trhsloc_Neither knifcuti,trhsloc_Neither pct_106,trhsloc_Neither othrweap,trhsloc_Neither pf_hcuff,trhsloc_Neither age,trhsloc_Neither pct_25,trhsloc_Neither descr_CPW,trhsloc_Neither sb_hdobj,descr_PETIT LARCENY^2,descr_PETIT LARCENY pistol,descr_PETIT LARCENY pct_13,descr_PETIT LARCENY pct_52,descr_PETIT LARCENY sb_outln,descr_PETIT LARCENY knifcuti,descr_PETIT LARCENY pct_106,descr_PETIT LARCENY othrweap,descr_PETIT LARCENY pf_hcuff,descr_PETIT LARCENY age,descr_PETIT LARCENY pct_25,descr_PETIT LARCENY descr_CPW,descr_PETIT LARCENY sb_hdobj,pistol^2,pistol pct_13,pistol pct_52,pistol sb_outln,pistol knifcuti,pistol pct_106,pistol othrweap,pistol pf_hcuff,pistol age,pistol pct_25,pistol descr_CPW,pistol sb_hdobj,pct_13^2,pct_13 pct_52,pct_13 sb_outln,pct_13 knifcuti,pct_13 pct_106,pct_13 othrweap,pct_13 pf_hcuff,pct_13 age,pct_13 pct_25,pct_13 descr_CPW,pct_13 sb_hdobj,pct_52^2,pct_52 sb_outln,pct_52 knifcuti,pct_52 pct_106,pct_52 othrweap,pct_52 pf_hcuff,pct_52 age,pct_52 pct_25,pct_52 descr_CPW,pct_52 sb_hdobj,sb_outln^2,sb_outln knifcuti,sb_outln pct_106,sb_outln othrweap,sb_outln pf_hcuff,sb_outln age,sb_outln pct_25,sb_outln descr_CPW,sb_outln sb_hdobj,knifcuti^2,knifcuti pct_106,knifcuti othrweap,knifcuti pf_hcuff,knifcuti age,knifcuti pct_25,knifcuti descr_CPW,knifcuti sb_hdobj,pct_106^2,pct_106 othrweap,pct_106 pf_hcuff,pct_106 age,pct_106 pct_25,pct_106 descr_CPW,pct_106 sb_hdobj,othrweap^2,othrweap pf_hcuff,othrweap age,othrweap pct_25,othrweap descr_CPW,othrweap sb_hdobj,pf_hcuff^2,pf_hcuff age,pf_hcuff pct_25,pf_hcuff descr_CPW,pf_hcuff sb_hdobj,age^2,age pct_25,age descr_CPW,age sb_hdobj,pct_25^2,pct_25 descr_CPW,pct_25 sb_hdobj,descr_CPW^2,descr_CPW sb_hdobj,sb_hdobj^2
0,0.0,1.0,0.0,180.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,180.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,

In [325]:
# Conduct Train Test split on new feature set
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(df_poly2, df_target, 
                                                                    random_state=1, test_size=0.2)

In [326]:
# Create the instance
grid_dt_poly = GridSearchCV(clf, dt_params, cv=10, scoring='f1', verbose=1, n_jobs=-1)

# Fit the decision tree classifier to the training data
grid_dt_poly.fit(X_train_poly, y_train_poly)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  1.6min finished


GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 15, 20],
                         'min_samples_leaf': [2, 5, 10],
                         'min_samples_split': [5, 7, 10]},
             scoring='f1', verbose=1)

In [327]:
# Make predictions for y values in the training set
dtc_grid_train_preds_poly = grid_dt_poly.best_estimator_.predict(X_train_poly)

# Make predictions for y values in the testing dataset
dtc_grid_test_preds_poly = grid_dt_poly.best_estimator_.predict(X_test_poly)

# Train Metrics
dtc_grid_train_accuracy_poly = metrics.accuracy_score(y_train_poly, dtc_grid_train_preds_poly)
dtc_grid_train_precision_poly = metrics.precision_score(y_train_poly, dtc_grid_train_preds_poly)
dtc_grid_train_recall_poly = metrics.recall_score(y_train_poly, dtc_grid_train_preds_poly)
dtc_grid_train_f1_poly = metrics.f1_score(y_train_poly, dtc_grid_train_preds_poly)

# Test Metrics
dtc_grid_accuracy_poly = metrics.accuracy_score(y_test_poly, dtc_grid_test_preds_poly)
dtc_grid_precision_poly = metrics.precision_score(y_test_poly, dtc_grid_test_preds_poly)
dtc_grid_recall_poly = metrics.recall_score(y_test_poly, dtc_grid_test_preds_poly)
dtc_grid_f1_poly = metrics.f1_score(y_test_poly, dtc_grid_test_preds_poly)


performance.append({'model': 'GridSearchCV Decision Tree Model w/ Poly Features', '# Features': X_train_poly.shape[1], 'Train Accuracy': dtc_grid_train_accuracy_poly,
                    'Test Accuracy': dtc_grid_accuracy_poly, 'Train Recall': dtc_grid_train_recall_poly,
                    'Test Recall': dtc_grid_recall_poly, 'Train Precision': dtc_grid_train_precision_poly,
                    'Test Precision': dtc_grid_precision_poly, 'Train F1': dtc_grid_train_f1_poly,
                    'Test F1': dtc_grid_f1_poly})

In [328]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952
6,Basic Decision Tree Selected Features,99,1.000000,0.836356,1.000000,0.676033,1.000000,0.660743,1.000000,0.668301
7,GridSearchCV Decision Tree Model w/ Selected F...,99,0.894891,0.868198,0.775450,0.735537,0.784987,0.727124,0.780190,0.731306
8,GridSearchCV Decision Tree Model w/ Feature Im...,66,0.895697,0.869811,0.775869,0.737190,0.787415,0.731148,0.781599,0.734156
9,Basic LogReg w/ Final Features & RFECV,29,0.858007,0.851270,0.772937,0.780165,0.680310,0.666667,0.723671,0.718964


In [329]:
# Reviewing feature importance
list_of_tuples = list(zip(list(df_poly2.columns), list(grid_dt_poly.best_estimator_.feature_importances_)))
poly_feat_perf = pd.DataFrame(list_of_tuples, columns = ['features', 'feature_importance'])

In [330]:
poly_feat_perf = poly_feat_perf.sort_values(by='feature_importance', ascending=False)

In [333]:
poly_feat_perf.reset_index(inplace=True)

In [335]:
pd.set_option('display.max_rows', 500)
poly_feat_perf

,index,features,feature_importance
0,116,weight searched,5.821202e-01
1,340,contrabn age,9.320754e-02
2,54,descr_CRIMINAL TRESPASS age,6.870072e-02
3,450,pf_hcuff age,3.613620e-02
4,13,contrabn,2.124799e-02
5,457,age sb_hdobj,1.739054e-02
6,125,weight trhsloc_Neither,1.362526e-02
7,432,knifcuti age,1.174297e-02
8,135,weight age,1.167958e-02
9,118,weight descr_THEFT OF SERVICES,1.039716e-02


In [336]:
# Re-running the model removing non-important features
poly_feat_importance = list(poly_feat_perf.features[:71])

In [337]:
final_features_poly = df_poly2.loc[:,poly_feat_importance]

In [338]:
# Conduct Train Test split on new feature set
X_train_poly_fin, X_test_poly_fin, y_train_poly_fin, y_test_poly_fin = train_test_split(final_features_poly, df_target, 
                                                                    random_state=1, test_size=0.2)

In [339]:
# Create the instance
fin_poly_grid = GridSearchCV(clf, dt_params, cv=10, scoring='f1', verbose=1, n_jobs=-1)

# Fit the decision tree classifier to the training data
fin_poly_grid.fit(X_train_poly_fin, y_train_poly_fin)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 780 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:   18.5s finished


GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 5, 10, 15, 20],
                         'min_samples_leaf': [2, 5, 10],
                         'min_samples_split': [5, 7, 10]},
             scoring='f1', verbose=1)

In [340]:
# Make predictions for y values in the training set
finpoly_train_preds = fin_poly_grid.best_estimator_.predict(X_train_poly_fin)

# Make predictions for y values in the testing dataset
finpoly_test_preds = fin_poly_grid.best_estimator_.predict(X_test_poly_fin)

# Train Metrics
finpoly_train_accuracy_temp = metrics.accuracy_score(y_train_poly_fin, finpoly_train_preds)
finpoly_train_precision_temp = metrics.precision_score(y_train_poly_fin, finpoly_train_preds)
finpoly_train_recall_temp = metrics.recall_score(y_train_poly_fin, finpoly_train_preds)
finpoly_train_f1_temp = metrics.f1_score(y_train_poly_fin, finpoly_train_preds)

# Test Metrics
finpoly_accuracy_temp = metrics.accuracy_score(y_test_poly_fin, finpoly_test_preds)
finpoly_precision_temp = metrics.precision_score(y_test_poly_fin, finpoly_test_preds)
finpoly_recall_temp = metrics.recall_score(y_test_poly_fin, finpoly_test_preds)
finpoly_f1_temp = metrics.f1_score(y_test_poly_fin, finpoly_test_preds)


performance.append({'model': 'Poly GridSearchCV Decision Tree Model w/ Feature Importance', '# Features': X_train_poly_fin.shape[1], 'Train Accuracy': finpoly_train_accuracy_temp,
                    'Test Accuracy': finpoly_accuracy_temp, 'Train Recall': finpoly_train_recall_temp,
                    'Test Recall': finpoly_recall_temp, 'Train Precision': finpoly_train_precision_temp,
                    'Test Precision': finpoly_precision_temp, 'Train F1': finpoly_train_f1_temp,
                    'Test F1': finpoly_f1_temp})

In [341]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952
6,Basic Decision Tree Selected Features,99,1.000000,0.836356,1.000000,0.676033,1.000000,0.660743,1.000000,0.668301
7,GridSearchCV Decision Tree Model w/ Selected F...,99,0.894891,0.868198,0.775450,0.735537,0.784987,0.727124,0.780190,0.731306
8,GridSearchCV Decision Tree Model w/ Feature Im...,66,0.895697,0.869811,0.775869,0.737190,0.787415,0.731148,0.781599,0.734156
9,Basic LogReg w/ Final Features & RFECV,29,0.858007,0.851270,0.772937,0.780165,0.680310,0.666667,0.723671,0.718964


## Recursive Feature Elimination with Logistic Regression
As of now, the most recent model decreased in performance, but what if we use RFECV to select the optimal number of features and take the top 15 again from grid search?

In [345]:
selector_poly = RFECV(LogisticRegression(class_weight='balanced', max_iter=10000), step=1, cv=10)
selector_poly.fit(X_train_poly, y_train_poly)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

KeyboardInterrupt: 

In [346]:
pd.DataFrame(performance)

,model,# Features,Train Accuracy,Test Accuracy,Train Recall,Test Recall,Train Precision,Test Precision,Train F1,Test F1
0,Basic Logistic Regression,231,0.861635,0.858525,0.801843,0.796694,0.680171,0.678873,0.736012,0.733080
1,GridSearchCV Logistic Regression,231,0.860425,0.855300,0.788018,0.796694,0.681522,0.674221,0.730911,0.726163
2,Basic Logistic Regression Selected,99,0.859216,0.858928,0.795140,0.793388,0.676408,0.680851,0.730984,0.732824
3,Basic LogReg w/ Selected Features & RFECV,15,0.860828,0.863362,0.760369,0.780165,0.691692,0.696165,0.724406,0.735776
4,Basic Decision Tree Model,231,1.000000,0.851270,1.000000,0.700826,1.000000,0.692810,1.000000,0.696795
5,GridSearchCV Decision Tree Model,231,0.905271,0.875050,0.803519,0.740496,0.802846,0.745424,0.803183,0.742952
6,Basic Decision Tree Selected Features,99,1.000000,0.836356,1.000000,0.676033,1.000000,0.660743,1.000000,0.668301
7,GridSearchCV Decision Tree Model w/ Selected F...,99,0.894891,0.868198,0.775450,0.735537,0.784987,0.727124,0.780190,0.731306
8,GridSearchCV Decision Tree Model w/ Feature Im...,66,0.895697,0.869811,0.775869,0.737190,0.787415,0.731148,0.781599,0.734156
9,Basic LogReg w/ Final Features & RFECV,29,0.858007,0.851270,0.772937,0.780165,0.680310,0.666667,0.723671,0.718964


Due to lack of time, I will not be able to run an RFECV model, so I'll choose the best performing model from above. 

**GridSearch Decision Tree Model with Best Performing Features and Non-zero feature importance (11)**
This modelruns the best on 24 features for enhanced interpretability. This model performs the best and most consistent across all metrics. 